<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/Symbol-to-Symbol_Attack_Clean-1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbol-to_Symbol Attack

# Loading Necesary Libraries

In [70]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

# Generate Train/Test data

In [71]:
def create_df(input_data_path, distances, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  X=None
  Y=[]
  colnames=['i'+str(i) for i in range(nsymbols)]

  for d in distances:
    dist=d*span_length
    if dist<min_dist or dist>max_dist: continue
    filename='consts_'+str(d)+'span.csv'
    df_aux=pd.read_csv(input_data_path+'/'+filename, sep=",", header=None)
    df_aux = df_aux.T
    df_aux.columns=colnames
    Y=Y+[dist]*df_aux.shape[0]
    if X is None: X=df_aux
    else: X=X.append(df_aux)
  return X, Y

In [72]:
def strToTuple(s):
    s_aux=s.split("i")
    s=s_aux[0]+"j"
    return complex(s)

In [73]:
def test_train_indexes_v1():
  train_idxs = []
  test_idxs = []
  for i in range(25):
      for j in range(25):
          train_idxs.append(50*i + j)
          test_idxs.append(50*(i+1)-1-j)
  return train_idxs, test_idxs

In [74]:
def test_train_indexes_v2():
  train_idxs2 = []
  test_idxs2 = []
  for i in range(25):
      for j in range(25):
          test_idxs2.append(50*(i+1)-1-j)

  for i in range(25):
    for j in range(2):
      train_idxs2.append(50*i + j)
  return train_idxs2, test_idxs2

# Utils

## Gradient Descent Function

In [75]:
def grad_b (M,m_og,m_tg,b,beta):
  return 2*beta*(M @ m_og + b - m_tg)

In [76]:
def grad_M(M,cov_og,cov_tg,alpha):
    error = M @ cov_og @ np.transpose(M) - cov_tg  

    m1 = (2*error[0][0] * (2*cov_og[0][0]*M[0][0] + 2*cov_og[0][1]*M[0][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[1][0] + cov_og[0][1]*M[1][1]))
    
    m2 = (2*error[0][0] * (2*cov_og[1][1]*M[0][1] + 2*cov_og[0][1]*M[0][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[1][0] + cov_og[1][1]*M[1][1]))
    
    m3 = (2*error[1][1] * (2*cov_og[0][0]*M[1][0] + 2*cov_og[0][1]*M[1][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[0][0] + cov_og[0][1]*M[0][1]))

    m4 = (2*error[1][1] * (2*cov_og[0][0]*M[1][1] + 2*cov_og[0][1]*M[1][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[0][0] + cov_og[1][1]*M[0][1]))
    
    return alpha*np.array([[m1, m2], [m3, m4]])

In [77]:
def gradient_descent(alpha,beta,m_tg,m_og,cov_tg,cov_og,nu,log):
  Ms = []
  bs = []
  M = np.random.rand(2,2)
  #M = np.array([[1, 0], [0, 1]])
  b = np.random.rand(2,1)
  #b = np.array([[0], [0]])
  #for i in range(100000):
  i = 0
  while True:
    # print('From:', M @ cov_source @ np.transpose(M))
    # print('To:', cov_target)
    L = loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b) 
    if  L < 1e-20:
        Ms.append(M)
        bs.append(b)
        break
    b = b - nu*grad_b(M,m_og,m_tg,b,beta)
    M = M - nu*grad_M(M,cov_og,cov_tg,alpha)
    i+= 1
    if (i>= 5000 and i <= 5025) or (i>= 5975 and i <= 6000):
      Ms.append(M)
      bs.append(b)
    if not i%5000 and log: print(L)
  if log: print("-"*25)
  return Ms,bs

## Compute parameters and mean/covariance

In [78]:
def compute_mean_and_cov(data):
  aux_x = [] # Reales
  aux_y = [] # Imag
  for obs in data:
    aux_x.append(obs[0])
    aux_y.append(obs[1])
  return np.array([[np.mean(aux_x)],[np.mean(aux_y)]]), np.cov(aux_x,aux_y)


def total_loss(m_mod,m_tg,cov_mod,cov_tg):
  return (sum(sum(np.power(cov_mod-cov_tg,2))) + sum(np.power(m_mod - m_tg,2)))[0]


def loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b):
  first = alpha*sum(sum(np.power(M @ cov_og @ np.transpose(M) - cov_tg, 2)))
  second = beta*sum(np.power(M @ m_og + b - m_tg, 2))
  a = first+second
  return a

In [79]:
def compute_parameters (const,method,source,target, params = None, log = False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  q_target = []

  for i in range(len(source)):

    q_source += [[x.real,x.imag] for x in source.values.tolist()[i] if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]
    q_target += [[x.real,x.imag] for x in target.values.tolist()[i]if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]

  unbiased = (len(q_source)/(len(q_source)-1) * 125/126)

  mean_source,cov_source = compute_mean_and_cov(q_source)
  mean_target, cov_target= compute_mean_and_cov(q_target)

  cov_source *= unbiased
  cov_target *= unbiased

  if method == "GD":
  
    alpha = params["alpha"] if params["alpha"] is not None and params["alpha"]>0 and params["alpha"]<=1 and params["alpha"]>params["beta"]  else 3/4
    beta = params["beta"] if params["beta"] is not None and params["beta"]>=0 and params["beta"]<1 and params["beta"]<params["alpha"]  else 1/4

    M, b = gradient_descent(alpha,beta,mean_target,mean_source,cov_target,cov_source, 0.5,log)
    return M,b

  elif method == "Z":
    return mean_source,cov_source,mean_target,cov_target

## Modify GD/Z

In [80]:
def modify_const_GD(const,source,M,b, target = None, return_plots=False):

  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]
      
  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  for mult in mults:
    res = (M @ mult + b).tolist()

    # --------- Old line ---------
    #new_points.append([res[0][0],res[1][0]])
    nested_check = any(isinstance(i, list) for i in res[0])
    if nested_check:
      new_points.append([res[0][0][0],res[0][1][0]])
    else:
      new_points.append([res[0][0],res[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      return new_points, q_target

  return new_points, indexes

In [81]:
def modify_const_Z(const,source,mean_source,cov_source,mean_target,cov_target,target = None, return_plots=False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]

  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  w, v = np.linalg.eig(cov_source)
  S1 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)
  w, v = np.linalg.eig(cov_target)
  S2 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)

  for mult in mults:
    normalized = np.linalg.inv(S1) @ (mult - mean_source)
    denormalized = S2 @ normalized + mean_target

    new_points.append([denormalized[0][0], denormalized[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      

  return new_points, indexes

## Compare plots

In [82]:
def generate_comparison_plot(mod_points,ptarget, return_plots=False):
  x1 = [x[0] for  x in mod_points]
  y1 = [x[1] for  x in mod_points]
  x2 = [x[0] for  x in ptarget]
  y2 = [x[1] for  x in ptarget]

  if return_plots:
      return go.Scatter(x = x1, y = y1, mode='markers'), go.Scatter(x = x2, y = y2,mode='markers') 

  fig = make_subplots(rows=1, cols=2)
  fig.add_trace(
    go.Scatter(x = x1, y = y1,mode='markers'),
    
    row=1, col=1
  )
  fig.add_trace(
    go.Scatter(x = x2, y = y2,mode='markers'),
    row=1, col=2
  )
  fig.update_layout(height=500, width=1000, title_text="Point comparison",autosize = False)
  fig.show()

## Distance Calc

In [83]:
def L2dist(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))

# Main

In [110]:
def mod_df_generator(df, output_filepath, raw_data = True, mod_ite = None):
  sample_arr = []
  symbol_arr = []
  real_arr = []
  imag_arr = []
  
  if raw_data != True:
    source_arr = []
    target_arr = []

    vals_df = df.iloc[:, 2:df.shape[1]]
    distances_df = df.iloc[:, [0, 1]]

    sample_cnt = 1
    symbol_cnt = 1
    
    
    
    for x in vals_df.values.tolist():
      real_arr.append(x.real)
      imag_arr.append(x.imag)

      sample_arr.append(sample_cnt)
      symbol_arr.append(symbol_cnt)
      symbol_cnt += 1
      sample_cnt += 1

      source_arr.append(distances_df['source_distance'].loc[distances_df.index[sample_cnt-1]])
      target_arr.append(distances_df['target_distance'].loc[distances_df.index[sample_cnt-1]])


    data = {'Source_Distance':source_arr, 'Target_Distance':target_arr, 'Sample_Id':sample_arr, 'Symbol_Id':symbol_arr, 'Real':real_arr, 'Imag':imag_arr}  
    
    mod_df = pd.DataFrame(data)
    
    filename = ('/consts_modified_{}span_mod.csv'.format(mod_ite))
   
  else:
    vals_df = df

    sample_cnt = 1
    file_cnt = 1
    for i in range(len(vals_df)):
      symbol_cnt = 1
      for x in vals_df.values.tolist()[i]:
        real_arr.append(x.real)
        imag_arr.append(x.imag)

        sample_arr.append(sample_cnt)
        symbol_arr.append(symbol_cnt)
        symbol_cnt += 1

      sample_cnt += 1
      if (i+1) % 50 == 0:
        data = {'Sample_Id':sample_arr, 'Symbol_Id':symbol_arr, 'Real':real_arr, 'Imag':imag_arr}  
        
        mod_df = pd.DataFrame(data)
        
        filename = ('/consts_{}span_mod.csv'.format(file_cnt))

        file_cnt += 1
        #mod_df.to_csv(str(output_filepath)+filename, index=False, encoding='utf-8-sig')      
        
        # Reset values
        sample_cnt = 1
        sample_arr = []
        symbol_arr = []
        real_arr = []
        imag_arr = [] 

In [112]:
print(F.shape)
display(F)

(75, 2050)


,source_distance,target_distance,i0,i1,i2,i3,i4,i5,i6,i7,...,i2038,i2039,i2040,i2041,i2042,i2043,i2044,i2045,i2046,i2047
0,80,2000,-2.877904+1.945910j,-2.866243+2.459064j,-3.563065+2.823870j,1.504892-2.636640j,3.026783+3.026363j,-2.969359-1.104804j,2.994048-2.887569j,0.604648+0.646593j,...,1.030606-1.022136j,2.933281-0.912271j,1.166157+1.495137j,3.077720-0.877666j,2.936849+3.057341j,-2.859879-3.058564j,0.987654+3.068215j,-0.947065-3.087723j,-3.445789+3.040006j,2.944331+3.114944j
1,80,2000,-3.618655+2.664759j,-3.649981+2.780303j,-3.845321+3.109076j,1.106185-2.562833j,2.866525+3.037062j,-2.976217-1.093614j,3.068700-2.994692j,0.480972+0.975449j,...,1.018686-1.043304j,2.980617-0.923241j,0.780439+1.321460j,2.897467-0.886151j,2.946413+3.054280j,-2.907779-3.032040j,0.916568+3.080998j,-0.967152-3.082439j,-3.351996+2.800125j,2.837659+3.121680j
2,80,2000,-3.168355+2.104893j,-3.397756+2.602836j,-4.151525+2.818827j,1.629795-3.067815j,2.959997+3.028488j,-2.791528-1.156530j,3.062512-2.973623j,0.647097+1.009172j,...,0.949409-0.953000j,2.963092-0.951192j,0.276099+0.802371j,2.996437-0.920273j,3.019396+2.996123j,-2.857622-3.027858j,0.988565+3.090861j,-0.920247-3.156104j,-3.046327+2.590407j,2.972642+2.978592j
3,80,2000,-3.461209+2.657069j,-3.840268+2.512949j,-3.159784+2.672637j,1.164968-2.433581j,2.855914+3.046808j,-2.902890-1.038207j,3.064385-2.927372j,0.814322+0.803847j,...,1.083347-1.050484j,3.098861-1.030020j,0.179399+1.007934j,3.066069-0.783748j,2.923210+3.108695j,-2.918734-3.071529j,0.850003+3.034759j,-0.909367-2.994824j,-3.144492+2.822334j,2.996194+3.065398j
4,80,2000,-3.204975+2.898114j,-3.021164+2.460120j,-3.528790+2.977125j,1.279275-2.958487j,2.813784+3.053663j,-3.044200-1.027796j,3.100371-2.875155j,0.024957+1.165572j,...,1.118578-0.909116j,2.934110-1.052446j,1.161666+1.109181j,2.969118-0.894978j,2.883089+3.130651j,-2.879680-3.061518j,0.995245+2.989870j,-0.844128-2.965608j,-3.442820+2.142697j,2.911049+3.156055j
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,240,2000,-2.891531+2.497288j,-3.134856+2.727263j,-3.180052+2.744371j,1.255845-2.678189j,2.987842+3.023532j,-2.836007-0.949170j,2.959554-2.800669j,0.426247+0.859036j,...,0.933137-0.898768j,2.884200-0.715913j,1.586574+1.288284j,2.818630-0.932364j,3.062784+2.942385j,-2.827252-2.938253j,0.924956+2.993245j,-1.105116-3.116271j,-3.208459+2.957074j,2.916134+3.003972j
71,240,2000,-2.915022+2.418869j,-2.787346+2.626541j,-3.275253+3.126156j,0.974624-2.552572j,3.029598+2.930374j,-2.930038-1.056641j,3.014486-2.898527j,0.401971+0.531870j,...,1.059652-0.871580j,2.965689-0.864824j,1.416313+1.330848j,2.959576-0.877139j,2.961272+2.943494j,-2.919405-2.886885j,1.030664+3.046456j,-0.963395-3.237141j,-3.396699+2.786296j,3.110396+3.031849j
72,240,2000,-3.078043+2.764804j,-3.225533+2.596509j,-3.549142+2.897572j,1.225620-2.786696j,3.026053+3.021865j,-2.882055-1.127848j,2.914688-2.907749j,0.346020+0.503236j,...,1.035255-0.832380j,2.886778-0.774848j,1.185388+0.770013j,2.883210-0.805473j,2.980132+3.111760j,-2.864364-3.023170j,1.141031+2.949808j,-1.070006-3.149225j,-3.442839+2.789320j,3.139453+2.986221j
73,240,2000,-2.984301+2.825730j,-3.096317+2.415385j,-3.049303+2.768068j,1.151928-2.848050j,2.896060+2.988814j,-2.935977-1.149665j,2.954241-2.863498j,0.741179+0.807679j,...,1.102956-0.831787j,2.907757-0.828482j,1.133974+1.473488j,2.978799-0.890712j,3.025123+2.928226j,-2.959750-2.877291j,0.984512+3.039499j,-1.022601-3.166842j,-3.278895+3.052995j,3.002278+3.033806j


In [85]:
'''
input_data_path=path+"/rawData"

distances=[i for i in range(1,26)]

# Obtain df for each distance (X) and  distance*nsymbols (Y)
Xs, Y = create_df(input_data_path, distances, nsymbols = 2048, min_dist = 0, max_dist = 3000)
Xs = Xs.applymap(strToTuple)
'''

'\ninput_data_path=path+"/rawData"\n\ndistances=[i for i in range(1,26)]\n\n# Obtain df for each distance (X) and  distance*nsymbols (Y)\nXs, Y = create_df(input_data_path, distances, nsymbols = 2048, min_dist = 0, max_dist = 3000)\nXs = Xs.applymap(strToTuple)\n'

In [108]:
def symbol2symbol_main(method, quadrants, input_data_path, distances, output_features, output_mod, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  
  # Get input data
  X, Y = create_df(input_data_path, distances, nsymbols, min_dist, max_dist)
  X = X.applymap(strToTuple)

  # Generate mod of the original file
  #mod_df_generator(X, input_data_path)
  modify_data_df = pd.DataFrame()

  train_idxs2, test_idxs2 = test_train_indexes_v2()

  X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
  X_test2 = X.iloc[test_idxs2].reset_index(drop = "True") 

  data_new = {}

  for i in range(0,25):
    data_new[str(80*(i+1))] = {"Train":X_train2.iloc[i*2:(i*2+2)].reset_index(drop = "True"),"Test":X_test2.iloc[i*25:(i*25+25)].reset_index(drop = "True")}  

  first = True

  for i in tqdm(range(25)):
      print('\n')
      print("i = ",i)
      for j in range(i+1,25):
          #print(" j = ",j)
          source = data_new[str(distances[i]*80)]['Train']
          target = data_new[str(distances[j]*80)]['Train']
          source_test = data_new[str(distances[i]*80)]['Test']

          #print('Source = {}'.format(str(distances[i]*80)))
          #print('Target = {}'.format(str(distances[j]*80)))
          source_test2 = source_test.copy()

          for quadrant in quadrants:
              #print("   quadrant = ", quadrant)
              if method == "GD":
                M, b = compute_parameters(quadrant, 'GD', source, target, params=params)
                new_points, indexes = modify_const_GD(quadrant, source_test, M, b)
              elif method == "Z":
                mean_source,cov_source,mean_target,cov_target = compute_parameters(quadrant, 'Z', source, target, params=None)
                new_points, indexes = modify_const_Z(quadrant,source_test,mean_source,cov_source,mean_target,cov_target)

              for k in range(len(indexes)):
                  source_test2.iloc[indexes[k][0],indexes[k][1]] = complex(new_points[k][0], new_points[k][1])

      
          old_df = source_test2.copy()
          old_df.insert(loc = 0,column = 'source_distance',value = str(distances[i]*80))
          old_df.insert(loc = 1,column = 'target_distance',value = str(distances[j]*80))
          display(old_df)
          modify_data_df  = pd.concat([modify_data_df, old_df], ignore_index=True)
          #new_mod_df = modify_data_df.append(source_test2)
          if i == 2:
            return modify_data_df

  
'''              
          #mod_df_generator(source_test2, output_mod, raw_data = False, mod_ite = i)
          F=[]
          for k in range(source_test2.shape[0]):
              data2=list(source_test2.iloc[k,:])
              data2=[[float(d.real), float(d.imag)] for d in data2]
              gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)
              mus=gmm.means_
              sigmas=gmm.covariances_

              features=[distances[i]*80, distances[j]*80]
              #features=[distances[j]*80]

              for z in selCP_pos:
                  mindist=None
                  k_inc=None
                  for w in range(16):
                      d=L2dist(mus[w],z)
                      if mindist is None or mindist>d:
                          mindist=d
                          k_inc=w
                  #print(k_inc)
                  #print(mus[k_inc])
                  covmat=np.concatenate(list(sigmas[k_inc])).ravel().tolist()
                  #features = [*features, *mus[k_inc], *[covmat[0], covmat[3]]]
                  features = [*features, *mus[k_inc], *covmat]
              F.append(features)
          #return F




          header=['original_dist', 'target_dist']
          
          for j in selCP:
              header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ri_'+str(j),'sigma_ir_'+str(j),'sigma_ii_'+str(j)]]
              #header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ii_'+str(j)]]
          with open(output_mod + output_features, 'a', encoding='UTF8', newline='') as f:
              writer = csv.writer(f)
              # write the header
              if first:
                writer.writerow(header)
                first = False
              # write multiple rows
              writer.writerows(F)
      # Here new code        
'''

"              \n          #mod_df_generator(source_test2, output_mod, raw_data = False)\n          F=[]\n          for k in range(source_test2.shape[0]):\n              data2=list(source_test2.iloc[k,:])\n              data2=[[float(d.real), float(d.imag)] for d in data2]\n              gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)\n              mus=gmm.means_\n              sigmas=gmm.covariances_\n\n              features=[distances[i]*80, distances[j]*80]\n              #features=[distances[j]*80]\n\n              for z in selCP_pos:\n                  mindist=None\n                  k_inc=None\n                  for w in range(16):\n                      d=L2dist(mus[w],z)\n                      if mindist is None or mindist>d:\n                          mindist=d\n                          k_inc=w\n                  #print(k_inc)\n                  #print(mus[k_inc])\n                  covmat=np.concatenate(list(sigmas[k_inc])).ravel().

#Run Main 

## Load Enviroment

In [86]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [87]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

## Global Variables

In [88]:
# Global variables

distances=[i for i in range(1,26)]
nsamples=50
span_length=80
nsymbols=2048

######
min_dist=0
max_dist=3000
selCP=[1,7,10,15]
selCP_pos=[(-3,3),(1,1),(-1,-1),(1,-3)]
my_centers=[[-3,3],[-1,3],[1,3],[3,3],[-3,1],[-1,1],[1,1],[3,1],[-3,-1],[-1,-1],[1,-1],[3,-1], [-3,-3],[-1,-3],[1,-3],[3,-3]]
######

## Execution Z Score

### User inputs

In [89]:
# PLEASE SELCET THE METHOD AND QUADRANTS THAT ARE DESIRED TO BE TESTED
method = "Z" # or method = "GD"
quadrants = [1, 7, 10, 15]

# PLEASE spicify the name of the new file
# Output features
output_features = "/Z_alpha_1.csv"

#Output constellation

#Output mod files
output_mod = path + '/modifiedData/'+ method

# PLEASE review that the path below is correct
input_data_path=path+"/rawData"
print('filepath selected: {}'.format(input_data_path))

filepath selected: /content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset/rawData


### Lauch

In [109]:
F = symbol2symbol_main(method, quadrants, input_data_path, distances, output_features, output_mod, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000)

  0%|          | 0/25 [00:00<?, ?it/s]



i =  0


,source_distance,target_distance,i0,i1,i2,i3,i4,i5,i6,i7,...,i2038,i2039,i2040,i2041,i2042,i2043,i2044,i2045,i2046,i2047
0,80,2000,-2.877904+1.945910j,-2.866243+2.459064j,-3.563065+2.823870j,1.504892-2.636640j,3.026783+3.026363j,-2.969359-1.104804j,2.994048-2.887569j,0.604648+0.646593j,...,1.030606-1.022136j,2.933281-0.912271j,1.166157+1.495137j,3.077720-0.877666j,2.936849+3.057341j,-2.859879-3.058564j,0.987654+3.068215j,-0.947065-3.087723j,-3.445789+3.040006j,2.944331+3.114944j
1,80,2000,-3.618655+2.664759j,-3.649981+2.780303j,-3.845321+3.109076j,1.106185-2.562833j,2.866525+3.037062j,-2.976217-1.093614j,3.068700-2.994692j,0.480972+0.975449j,...,1.018686-1.043304j,2.980617-0.923241j,0.780439+1.321460j,2.897467-0.886151j,2.946413+3.054280j,-2.907779-3.032040j,0.916568+3.080998j,-0.967152-3.082439j,-3.351996+2.800125j,2.837659+3.121680j
2,80,2000,-3.168355+2.104893j,-3.397756+2.602836j,-4.151525+2.818827j,1.629795-3.067815j,2.959997+3.028488j,-2.791528-1.156530j,3.062512-2.973623j,0.647097+1.009172j,...,0.949409-0.953000j,2.963092-0.951192j,0.276099+0.802371j,2.996437-0.920273j,3.019396+2.996123j,-2.857622-3.027858j,0.988565+3.090861j,-0.920247-3.156104j,-3.046327+2.590407j,2.972642+2.978592j
3,80,2000,-3.461209+2.657069j,-3.840268+2.512949j,-3.159784+2.672637j,1.164968-2.433581j,2.855914+3.046808j,-2.902890-1.038207j,3.064385-2.927372j,0.814322+0.803847j,...,1.083347-1.050484j,3.098861-1.030020j,0.179399+1.007934j,3.066069-0.783748j,2.923210+3.108695j,-2.918734-3.071529j,0.850003+3.034759j,-0.909367-2.994824j,-3.144492+2.822334j,2.996194+3.065398j
4,80,2000,-3.204975+2.898114j,-3.021164+2.460120j,-3.528790+2.977125j,1.279275-2.958487j,2.813784+3.053663j,-3.044200-1.027796j,3.100371-2.875155j,0.024957+1.165572j,...,1.118578-0.909116j,2.934110-1.052446j,1.161666+1.109181j,2.969118-0.894978j,2.883089+3.130651j,-2.879680-3.061518j,0.995245+2.989870j,-0.844128-2.965608j,-3.442820+2.142697j,2.911049+3.156055j
5,80,2000,-3.536324+2.729436j,-2.826621+2.536322j,-3.253443+3.332075j,0.995978-2.916248j,2.956546+2.987119j,-2.950761-1.165853j,3.173992-2.910556j,0.380140+0.714411j,...,0.997758-1.039745j,3.044767-0.882087j,1.000232+1.666399j,2.886551-0.843001j,3.019924+2.992457j,-2.863967-3.142740j,1.088269+2.934255j,-0.963003-3.087128j,-3.142609+2.934396j,2.892579+3.071078j
6,80,2000,-3.780853+2.676007j,-3.007551+2.678963j,-3.604257+3.569052j,1.245722-2.848402j,2.842408+2.993903j,-2.896300-1.093272j,3.112511-2.840368j,0.490212+1.209451j,...,1.045421-1.048230j,2.994297-0.936755j,0.576010+1.202964j,2.942416-0.866063j,2.865438+3.147579j,-2.883983-2.958881j,1.002043+3.040390j,-0.948962-3.043690j,-2.989675+2.557486j,3.049469+3.125919j
7,80,2000,-3.315113+2.694108j,-3.437521+2.787339j,-3.499909+2.895431j,1.345358-2.723243j,2.952608+3.190000j,-2.865174-1.086075j,3.037258-2.921921j,0.868908+0.576568j,...,1.094264-1.152656j,3.034982-0.980455j,1.708556+0.931784j,2.977605-0.964947j,2.915813+3.154856j,-2.878824-3.053926j,0.945890+2.981577j,-0.875841-3.079029j,-2.969874+2.429478j,2.964730+3.069953j
8,80,2000,-3.494214+2.845239j,-3.685905+2.739029j,-3.496014+2.856649j,1.352474-2.715458j,2.861949+3.034223j,-2.920400-1.065227j,3.116617-2.868061j,1.047265+1.943918j,...,0.931463-1.001832j,3.038209-0.891187j,1.096269+1.470426j,2.957961-0.825850j,2.923749+3.170281j,-2.831101-3.111160j,0.988738+3.032605j,-0.867736-3.040001j,-3.344608+3.083113j,2.955872+3.122352j
9,80,2000,-3.483746+2.387656j,-3.140546+2.327934j,-3.340885+2.611897j,1.110923-3.698726j,3.046268+2.951372j,-3.023021-1.089708j,3.049732-2.891779j,0.091462+1.196741j,...,0.989591-1.003984j,2.913720-0.978003j,0.781066+1.525080j,2.924198-0.926191j,2.853057+2.977592j,-2.965142-3.100775j,0.917134+3.002135j,-0.886928-3.026397j,-3.196108+2.762565j,2.914253+3.134684j


  4%|▍         | 1/25 [00:38<15:15, 38.13s/it]



i =  1


,source_distance,target_distance,i0,i1,i2,i3,i4,i5,i6,i7,...,i2038,i2039,i2040,i2041,i2042,i2043,i2044,i2045,i2046,i2047
0,160,2000,-3.021655+2.413267j,-3.026321+2.642813j,-3.420068+2.823853j,1.261831-2.905493j,2.960114+3.051458j,-2.839559-1.173948j,2.960961-2.834516j,0.447158+0.615481j,...,1.046856-0.925809j,2.964817-0.860123j,1.147861+1.205567j,2.939899-0.819555j,3.036203+3.012395j,-2.835753-3.000148j,1.023956+3.001420j,-1.025045-3.150028j,-3.271068+2.902088j,3.041272+3.028950j
1,160,2000,-3.019182+1.988424j,-2.779053+2.799424j,-3.327989+2.632172j,1.465169-2.539259j,2.951469+3.130780j,-2.895918-1.136734j,3.070773-2.816816j,0.410194+0.977258j,...,1.048421-0.995199j,3.001237-0.971607j,1.131394+1.140756j,3.075276-0.909533j,2.849853+3.059090j,-2.833266-2.998556j,0.966332+3.040834j,-1.080392-3.089020j,-3.238499+2.925852j,3.003649+3.083179j
2,160,2000,-3.334016+2.434500j,-2.816232+2.978878j,-4.239987+2.969051j,1.482420-2.446835j,2.966697+3.152428j,-2.936879-1.267141j,2.986886-2.789063j,0.491426+0.637241j,...,1.021739-1.000093j,2.962100-0.992816j,1.236492+1.551180j,3.074995-0.852435j,3.018812+3.123903j,-2.892604-3.007152j,0.989782+3.024056j,-1.038801-3.104345j,-2.924626+2.813386j,3.013727+3.195677j
3,160,2000,-2.932318+2.357146j,-2.816317+2.954017j,-3.730736+3.000989j,1.256471-2.569372j,3.105664+3.128270j,-2.901697-1.191471j,3.078311-2.845483j,0.476681+0.729248j,...,1.009542-1.111058j,2.969423-0.835150j,1.198819+1.254501j,3.027734-0.723106j,2.998346+3.167875j,-2.878279-3.076559j,0.942529+3.117678j,-0.942845-3.084750j,-3.101766+2.671495j,2.945232+3.048444j
4,160,2000,-2.938344+2.200011j,-2.970814+3.086769j,-3.743591+3.209066j,1.705428-2.900371j,2.981406+3.010778j,-2.966653-1.136956j,2.996959-2.885236j,0.253173+1.194241j,...,1.067557-1.081597j,3.033710-0.784695j,1.089847+1.178624j,3.113797-0.792913j,2.902602+3.028900j,-2.923918-3.086432j,0.975547+3.048473j,-0.914134-3.039122j,-3.009600+2.683118j,3.008157+3.150454j
5,160,2000,-2.882812+2.164465j,-2.688690+2.782414j,-3.336458+3.028889j,1.409675-2.798226j,3.006272+3.204527j,-2.968582-1.255073j,2.972367-2.843217j,0.390692+1.294239j,...,1.008313-0.914704j,2.984450-0.927553j,1.282276+1.080303j,2.992383-0.880728j,3.040223+3.189241j,-2.968890-2.957314j,0.844359+3.132032j,-0.908625-2.955301j,-3.033215+3.242170j,2.960675+3.095611j
6,160,2000,-2.886404+2.039685j,-2.800993+2.893254j,-3.771654+2.833541j,1.439314-2.945048j,2.958489+3.042434j,-2.945680-1.134629j,3.069235-2.829572j,0.422873+1.556974j,...,1.005608-1.049597j,3.049018-0.877995j,0.598578+1.308611j,2.997280-0.870313j,2.927321+3.175886j,-2.859661-3.081779j,0.923046+3.091376j,-0.953708-3.106299j,-3.336137+2.777890j,3.034134+3.009103j
7,160,2000,-2.909911+2.025770j,-3.107441+2.879512j,-3.684783+3.093007j,1.096190-2.303423j,2.902225+3.089215j,-2.970714-1.166361j,3.004019-2.885090j,0.579223+0.751072j,...,1.009573-1.012787j,3.003403-0.917185j,0.939662+1.515293j,3.037070-0.853648j,2.892656+2.927899j,-2.965965-3.047009j,0.880458+3.090768j,-1.030764-3.118210j,-2.989714+3.015925j,3.054982+3.124133j
8,160,2000,-2.774271+1.942743j,-2.981955+3.038343j,-3.423319+3.294264j,1.564752-2.734727j,3.091524+3.147828j,-2.929897-1.105493j,3.091032-2.925613j,0.416544+0.611757j,...,0.972313-0.934935j,2.928988-0.850558j,0.632862+1.191067j,2.934520-0.822125j,2.875994+2.976415j,-2.940194-3.002514j,0.946061+2.981455j,-0.992232-3.129158j,-3.344528+3.189606j,2.994827+3.078739j
9,160,2000,-2.704746+2.304012j,-3.387230+2.532931j,-3.645931+2.896386j,1.488774-2.808605j,2.928395+3.031735j,-2.985651-1.220954j,3.010220-2.941451j,0.869282+0.634835j,...,1.043606-0.951734j,2.864460-0.860470j,1.175808+0.906442j,3.006671-0.909555j,2.980559+3.056960j,-2.938609-2.992836j,0.963044+2.960602j,-0.920613-3.035464j,-3.266709+2.414720j,2.993531+3.072819j


  8%|▊         | 2/25 [01:24<16:22, 42.71s/it]



i =  2


,source_distance,target_distance,i0,i1,i2,i3,i4,i5,i6,i7,...,i2038,i2039,i2040,i2041,i2042,i2043,i2044,i2045,i2046,i2047
0,240,2000,-3.030666+2.374972j,-3.332816+2.601395j,-3.582492+2.593031j,1.262570-2.903722j,3.040821+3.093713j,-2.890352-1.211233j,2.948465-2.791532j,0.370640+0.734696j,...,0.948631-0.912559j,2.960363-0.842051j,1.127485+1.372766j,2.906174-0.892432j,2.992534+2.927509j,-2.964058-2.902310j,1.125410+2.998507j,-1.026434-3.113538j,-3.317870+3.083204j,3.003254+3.078467j
1,240,2000,-3.197229+2.499361j,-3.057663+2.689577j,-3.425325+2.938052j,1.062075-2.792460j,2.970314+3.018848j,-2.845300-1.093418j,3.034726-2.782848j,0.628197+0.813070j,...,1.007793-0.897203j,2.913290-0.752059j,1.447322+1.420222j,2.985230-0.910999j,2.963000+3.011299j,-2.844684-3.001904j,1.028990+2.969500j,-1.077947-3.134566j,-3.461314+3.023840j,3.032685+2.990025j
2,240,2000,-2.732823+2.276470j,-2.996769+2.521318j,-3.350518+2.695532j,1.095660-2.582220j,3.023392+3.029505j,-2.899506-1.181653j,2.875092-2.807396j,0.657658+0.661633j,...,1.022794-0.943364j,3.070882-0.710030j,1.309915+1.331397j,2.897188-0.805395j,2.954426+2.893342j,-2.875791-2.972817j,1.015758+3.067492j,-1.089217-3.148270j,-3.603839+3.185435j,2.977400+3.077170j
3,240,2000,-2.660924+2.464066j,-3.160560+2.849683j,-3.372238+3.038191j,1.337680-2.827987j,2.929764+2.922891j,-3.010815-1.066149j,3.016728-2.823224j,0.549723+0.580217j,...,1.060191-0.956099j,2.951905-0.815126j,1.120322+1.446981j,2.845320-0.905670j,3.068471+2.964658j,-3.054472-2.905367j,1.026154+3.091274j,-1.166790-3.135331j,-3.210551+3.161819j,3.167439+3.073778j
4,240,2000,-2.868553+2.629418j,-2.687676+2.593553j,-3.663607+2.926661j,1.346629-2.730425j,2.917513+2.886363j,-2.956699-1.069317j,2.887581-2.812660j,0.377538+0.986734j,...,0.938274-0.857511j,2.950557-0.749434j,1.261329+1.426533j,2.955401-0.854856j,3.000746+3.009040j,-2.955515-3.036149j,0.897607+2.978159j,-1.035018-3.111078j,-3.107344+2.800136j,3.008240+3.100861j
5,240,2000,-2.936745+2.412100j,-3.052008+2.441188j,-3.604278+2.724590j,1.170744-2.718346j,2.921534+2.947896j,-2.799831-1.067480j,2.961237-2.902168j,0.525922+0.284803j,...,1.075980-0.816309j,2.890887-0.811423j,1.107561+1.060464j,2.924360-0.886758j,3.004934+2.983440j,-2.927266-2.933820j,1.118517+3.063585j,-1.007565-3.209601j,-3.193410+2.866630j,3.044610+3.086007j
6,240,2000,-3.003992+2.626714j,-3.118133+2.454270j,-3.764437+2.960924j,1.192562-2.929953j,2.970581+2.985533j,-2.930276-1.030424j,2.927563-2.945100j,0.640908+0.672783j,...,1.007951-0.862325j,2.892611-0.806898j,1.218448+1.344616j,2.801638-0.877226j,2.941288+2.846397j,-2.951390-2.963376j,1.037453+3.036130j,-1.137981-3.232778j,-3.398288+2.886134j,3.035254+2.993093j
7,240,2000,-2.840657+2.592417j,-3.298055+2.826382j,-3.508143+2.869153j,1.166256-2.609814j,3.103511+3.083673j,-2.888896-1.089869j,2.833330-2.885450j,0.572695+0.652764j,...,0.930420-0.952906j,2.934994-0.837632j,1.247107+1.325756j,2.896393-0.857358j,2.993948+3.054747j,-2.840734-3.012378j,1.031873+3.061512j,-1.125103-3.169325j,-3.402178+2.639257j,3.004196+3.042385j
8,240,2000,-2.857174+2.570636j,-3.155110+2.760071j,-3.538692+2.657655j,1.071829-2.874454j,2.983043+3.014231j,-2.895584-0.994253j,2.980963-2.950168j,0.840899+0.853894j,...,1.039229-0.907303j,3.017601-0.771298j,1.411569+1.462681j,3.023427-0.788408j,2.929227+3.052832j,-2.870412-2.978918j,1.085884+3.024150j,-1.134057-3.260650j,-3.496083+3.009818j,3.016750+3.151942j
9,240,2000,-3.239494+2.385645j,-3.119948+2.656371j,-3.220706+2.876015j,1.149597-2.533707j,3.033523+2.952668j,-2.847500-1.004582j,2.931753-2.915647j,0.503061+0.793495j,...,1.027063-0.951795j,2.910111-0.865152j,1.354230+1.211489j,2.898779-0.973815j,2.966920+2.999381j,-2.835036-2.944793j,1.014803+2.951539j,-0.952302-3.049286j,-3.330361+2.689018j,3.042154+3.170248j


  8%|▊         | 2/25 [02:05<23:59, 62.59s/it]


In [91]:
print(F)

None


In [92]:
print(len(F[0]))

TypeError: ignored

In [ ]:
print(len(F))

In [111]:
print(F.shape)
display(F)

(75, 2050)


,source_distance,target_distance,i0,i1,i2,i3,i4,i5,i6,i7,...,i2038,i2039,i2040,i2041,i2042,i2043,i2044,i2045,i2046,i2047
0,80,2000,-2.877904+1.945910j,-2.866243+2.459064j,-3.563065+2.823870j,1.504892-2.636640j,3.026783+3.026363j,-2.969359-1.104804j,2.994048-2.887569j,0.604648+0.646593j,...,1.030606-1.022136j,2.933281-0.912271j,1.166157+1.495137j,3.077720-0.877666j,2.936849+3.057341j,-2.859879-3.058564j,0.987654+3.068215j,-0.947065-3.087723j,-3.445789+3.040006j,2.944331+3.114944j
1,80,2000,-3.618655+2.664759j,-3.649981+2.780303j,-3.845321+3.109076j,1.106185-2.562833j,2.866525+3.037062j,-2.976217-1.093614j,3.068700-2.994692j,0.480972+0.975449j,...,1.018686-1.043304j,2.980617-0.923241j,0.780439+1.321460j,2.897467-0.886151j,2.946413+3.054280j,-2.907779-3.032040j,0.916568+3.080998j,-0.967152-3.082439j,-3.351996+2.800125j,2.837659+3.121680j
2,80,2000,-3.168355+2.104893j,-3.397756+2.602836j,-4.151525+2.818827j,1.629795-3.067815j,2.959997+3.028488j,-2.791528-1.156530j,3.062512-2.973623j,0.647097+1.009172j,...,0.949409-0.953000j,2.963092-0.951192j,0.276099+0.802371j,2.996437-0.920273j,3.019396+2.996123j,-2.857622-3.027858j,0.988565+3.090861j,-0.920247-3.156104j,-3.046327+2.590407j,2.972642+2.978592j
3,80,2000,-3.461209+2.657069j,-3.840268+2.512949j,-3.159784+2.672637j,1.164968-2.433581j,2.855914+3.046808j,-2.902890-1.038207j,3.064385-2.927372j,0.814322+0.803847j,...,1.083347-1.050484j,3.098861-1.030020j,0.179399+1.007934j,3.066069-0.783748j,2.923210+3.108695j,-2.918734-3.071529j,0.850003+3.034759j,-0.909367-2.994824j,-3.144492+2.822334j,2.996194+3.065398j
4,80,2000,-3.204975+2.898114j,-3.021164+2.460120j,-3.528790+2.977125j,1.279275-2.958487j,2.813784+3.053663j,-3.044200-1.027796j,3.100371-2.875155j,0.024957+1.165572j,...,1.118578-0.909116j,2.934110-1.052446j,1.161666+1.109181j,2.969118-0.894978j,2.883089+3.130651j,-2.879680-3.061518j,0.995245+2.989870j,-0.844128-2.965608j,-3.442820+2.142697j,2.911049+3.156055j
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,240,2000,-2.891531+2.497288j,-3.134856+2.727263j,-3.180052+2.744371j,1.255845-2.678189j,2.987842+3.023532j,-2.836007-0.949170j,2.959554-2.800669j,0.426247+0.859036j,...,0.933137-0.898768j,2.884200-0.715913j,1.586574+1.288284j,2.818630-0.932364j,3.062784+2.942385j,-2.827252-2.938253j,0.924956+2.993245j,-1.105116-3.116271j,-3.208459+2.957074j,2.916134+3.003972j
71,240,2000,-2.915022+2.418869j,-2.787346+2.626541j,-3.275253+3.126156j,0.974624-2.552572j,3.029598+2.930374j,-2.930038-1.056641j,3.014486-2.898527j,0.401971+0.531870j,...,1.059652-0.871580j,2.965689-0.864824j,1.416313+1.330848j,2.959576-0.877139j,2.961272+2.943494j,-2.919405-2.886885j,1.030664+3.046456j,-0.963395-3.237141j,-3.396699+2.786296j,3.110396+3.031849j
72,240,2000,-3.078043+2.764804j,-3.225533+2.596509j,-3.549142+2.897572j,1.225620-2.786696j,3.026053+3.021865j,-2.882055-1.127848j,2.914688-2.907749j,0.346020+0.503236j,...,1.035255-0.832380j,2.886778-0.774848j,1.185388+0.770013j,2.883210-0.805473j,2.980132+3.111760j,-2.864364-3.023170j,1.141031+2.949808j,-1.070006-3.149225j,-3.442839+2.789320j,3.139453+2.986221j
73,240,2000,-2.984301+2.825730j,-3.096317+2.415385j,-3.049303+2.768068j,1.151928-2.848050j,2.896060+2.988814j,-2.935977-1.149665j,2.954241-2.863498j,0.741179+0.807679j,...,1.102956-0.831787j,2.907757-0.828482j,1.133974+1.473488j,2.978799-0.890712j,3.025123+2.928226j,-2.959750-2.877291j,0.984512+3.039499j,-1.022601-3.166842j,-3.278895+3.052995j,3.002278+3.033806j


## Execution Gradient Descent

### User Inputs

In [ ]:
# PLEASE SELCET THE METHOD AND QUADRANTS THAT ARE DESIRED TO BE TESTED
method = "GD" # or method = "Z"
quadrants = [1, 7, 10, 15]

# PLEASE spicify the name of the new file
output_features = "/GD_alpha_1.csv"

# PLEASE review that the path below is correct
input_data_path=path+"/rawData"
print('filepath selected: {}'.format(input_data_path))

#Output mod files
output_mod = path + '/modifiedData/'+ method

# Params for gradient descent alpha beta
a = 0.7
b = 0.2
params = {"alpha": float(a), "beta": float(b)}

### Lauch

In [ ]:
#symbol2symbol_main(method, quadrants, input_data_path, distances, output_features = output_filename, output_mod = output_mod, params = params, nsymbols = 2048, min_dist = 0, max_dist = 3000)